In [1]:
import os

#import pandas as pd
import numpy as np

from openai import OpenAI
from pymongo import MongoClient
from monggregate import Pipeline

In [10]:
openai_client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
)

model_name = "text-embedding-3-large"
max_dimensions = 2048

def _get_embedding(text:str, model:str=model_name):
   text = text.replace("\n", " ")
   return openai_client.embeddings.create(input = [text], model=model, dimensions=max_dimensions).data[0]

def get_embedding(text:str, model_name:str=model_name):
    try:
        return _get_embedding(text, model_name).embedding
    except Exception as e:
        print(e)
        return np.nan

In [11]:
mongo_client = MongoClient(os.environ["MONGODB_URI"])
db = mongo_client["spotify"]
collection = db["samples"]

In [12]:
indexes = ["lyrics_semantics_euclidean","lyrics_semantics_cosine", "lyrics_semantics_dot_product"]
def find_songs(text:str, limit:int=5, index:str = "lyrics_semantics_euclidean")->list[dict]:

    pipeline = Pipeline()
    pipeline.vector_search(
        index=index,
        path="embedding",
        query_vector=get_embedding(text),
        limit=limit,
        num_candidates=100
        ).project(
            projection=
            {
                "_id": 0, 
                #"text": 1, 
                "artist": 1, 
                "title": 1,
                "theme": 1,
                "score":{ 
                    "$meta": "vectorSearchScore" 
                    }
            }
        ).sort(
            descending="score"
        )

    return list(collection.aggregate(pipeline.export()))

In [13]:
def print_results(results:list[dict]):
    for result in results:
        print(f"{result['artist']}: {result['title']} - Score: {result['score']} - Theme: {result['theme']}")
        # print("---------------------")
        # print(result["text"])
        # print("=====================")

In [14]:
themes_prompts_map ={
    "Overcoming adversity":"The protagonist faces a seemingly insurmountable challenge but finds the strength to persevere, ultimately achieving a sense of triumph or personal growth.",
    "Lost love and longing":"The heartache and lingering emotions of a lost love, conveying a sense of nostalgia, yearning, and bittersweet acceptance",
    "Unconventional love":"Celebrates love that defies societal expectations or challenges traditional norms, highlighting the power of acceptance and embracing differences.",
    "Celebrating life's simple joys":"Pure happiness and appreciation for the beauty and wonder of everyday life, highlighting the simple moments that bring joy and fulfillment.",
    "Finding Hope in Unexpected Places":"Comfort and solace in the face of despair, highlighting the unexpected sources of hope and strength that can emerge even in difficult times",
    "The Beauty and Pain of Vulnerability":"Emotional complexities of vulnerability, exploring themes of opening oneself up to love, loss, and connection, despite the potential for pain.",
    "The Enduring Power of Love and Connection":"The enduring power of love and connection, exploring themes of friendship, romantic love, familial bonds, and the human yearning for belonging.",
    "Hustle and Ambition":"The relentless drive and ambition often found in urban communities, where overcoming challenges and achieving success are major motivators. Overcoming hardships, chasing dreams, and building empires from the ground up.",
    "Vulnerability and Emotional Honesty":"Heartbreak, self-doubt, mental health struggles, and finding strength in raw honesty",
    "Finding Joy and Celebration in Life's Moments":"Positivity, and joy, living life to the fullest, finding happiness in the simple things, and celebrating community and personal achievements."
}

In [18]:
for key, value in themes_prompts_map.items():
    print(f"Theme: {key}")
    print("---------------------")
    results = find_songs(value, limit=3, index=indexes[2])
    print_results(results)
    print("=====================")


Theme: Overcoming adversity
---------------------
Eminem: Lose Yourself - Score: 0.6477552056312561 - Theme: Overcoming adversity
Coldplay: Fix You - Score: 0.6138159036636353 - Theme: Finding Hope in Unexpected Places
Gloria Gaynor: I Will Survive - Score: 0.6045676469802856 - Theme: Overcoming adversity
Theme: Lost love and longing
---------------------
Taylor Swift: All Too Well - Score: 0.7166137099266052 - Theme: The Beauty and Pain of Vulnerability
Adele: Someone Like You - Score: 0.715863049030304 - Theme: Lost love and longing
Whitney Houston: I Will Always Love You - Score: 0.6894090175628662 - Theme: The Enduring Power of Love and Connection
Theme: Unconventional love
---------------------
Lady Gaga: Born This Way - Score: 0.6516531109809875 - Theme: Unconventional love
Pharrell Williams: Happy - Score: 0.5974611043930054 - Theme: Finding Joy and Celebration in Life's Moments
Coldplay: A Sky Full Of Stars - Score: 0.5889706611633301 - Theme: Unconventional love
Theme: Celebra